In [126]:
import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline
import pickle
import os
import sklearn.tree 
from sklearn.tree import DecisionTreeClassifier
import h2o
import random

In [1]:
from sklearn import metrics

In [176]:
h2o.init()

H2O cluster uptime:,20 hours 48 minutes 52 seconds 875 milliseconds
H2O cluster version:,3.2.0.9
H2O cluster name:,me
H2O cluster total nodes:,1
H2O cluster total memory:,1.98 GB
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [8]:
#define file path
file_path='e:/nycds/webscraping project/price_na_drp.csv'

In [11]:
#load data by using dictionary for params setting 
params_load_data={'path':file_path, 
                  'destination_frame':'prcnadrp','header':1,'sep':','}
prcnadrp=h2o.import_file(**params_load_data)



Parse Progress: [##################################################] 100%
Imported e:/nycds/webscraping project/price_na_drp.csv. Parsed 2,844 rows and 12 cols


In [10]:
#count number of records
num_records=data_hex.nrow

#print the number with comment 
print 'Records number in this data set is :'+str(num_records)

Records number in this data set is :2844


In [12]:
prcnadrp.head

H2OFrame with 2844 rows and 12 columns: 
   C1  Unnamed: 0               street address        city state  zip code  \
0   1           1    2907 East South Boulevard  Montgomery    AL     36116   
1   2           2          2150 Douglas Avenue     Brewton    AL     36426   
2   3           3                 254 W Bypass   Andalusia    AL     36420   
3   4           4  2608 West Meighan Boulevard     Gadsden    AL     35904   
4   5           5  2490 Schillinger Road South      Mobile    AL     36695   
5   8           8            5651 Endeavor Way      Tanner    AL     35671   
6   9           9            5860 Endeavor Way      Tanner    AL     35671   
7  10          10        1524 East 10th Street    Anniston    AL     36207   
8  11          11          5510 McFarland Blvd   Northport    AL     35476   
9  12          12            5950 Endeavor Way      Tanner    AL     35671   

  listing price                                      property name  \
0    $2,945,000  One Center - 

<bound method H2OFrame.head of >

In [17]:
prcnadrp.names

[u'',
 u'Unnamed: 0',
 u'street address',
 u'city',
 u'state',
 u'zip code',
 u'listing price',
 u'property name',
 u'property type',
 u'size sf',
 u'property class',
 u'cap rate']

In [19]:
prcnadrp=prcnadrp.drop('')

H2OFrame with 2844 rows and 11 columns: 
   Unnamed: 0               street address        city state  zip code  \
0           1    2907 East South Boulevard  Montgomery    AL     36116   
1           2          2150 Douglas Avenue     Brewton    AL     36426   
2           3                 254 W Bypass   Andalusia    AL     36420   
3           4  2608 West Meighan Boulevard     Gadsden    AL     35904   
4           5  2490 Schillinger Road South      Mobile    AL     36695   
5           8            5651 Endeavor Way      Tanner    AL     35671   
6           9            5860 Endeavor Way      Tanner    AL     35671   
7          10        1524 East 10th Street    Anniston    AL     36207   
8          11          5510 McFarland Blvd   Northport    AL     35476   
9          12            5950 Endeavor Way      Tanner    AL     35671   

  listing price                                      property name  \
0    $2,945,000  One Center - $30MM Anchors Underway. 20% Final...   
1   

In [21]:
prcnadrp=prcnadrp.drop('Unnamed: 0')

In [24]:
prcnadrp=prcnadrp.drop('C1')

In [101]:
prcnadrp=prcnadrp.drop('target')

In [26]:
prcnadrp.head

H2OFrame with 2844 rows and 10 columns: 
                street address        city state  zip code listing price  \
0    2907 East South Boulevard  Montgomery    AL     36116    $2,945,000   
1          2150 Douglas Avenue     Brewton    AL     36426    $1,577,901   
2                 254 W Bypass   Andalusia    AL     36420    $1,933,996   
3  2608 West Meighan Boulevard     Gadsden    AL     35904    $4,406,000   
4  2490 Schillinger Road South      Mobile    AL     36695    $5,085,714   
5            5651 Endeavor Way      Tanner    AL     35671    $5,216,200   
6            5860 Endeavor Way      Tanner    AL     35671    $5,749,800   
7        1524 East 10th Street    Anniston    AL     36207    $2,690,000   
8          5510 McFarland Blvd   Northport    AL     35476    $3,950,000   
9            5950 Endeavor Way      Tanner    AL     35671    $7,358,400   

                                       property name    property type  \
0  One Center - $30MM Anchors Underway. 20% Final

<bound method H2OFrame.head of >

In [102]:
#assign response and features variables 
#response variable 
response=unicode('property type')

#column names 
features=prcnadrp.names

In [103]:
#find the index of response variable 
for idx, feature in enumerate(features):
    if feature == response:
        idx_response=idx


In [104]:
#count number of features 
num_features_all=len(features)

In [105]:
#remove response 
features.remove(response)

#count number of resulting features
num_features=len(features)

#print with description 
print 'Number of features BEFORE removing response is :'+str(num_features_all)
print 'Number of features After removing response is : '+str(num_features)

Number of features BEFORE removing response is :10
Number of features After removing response is : 9


In [106]:
#splitting the data set
# creating a vector with uniformly distributed numbers between 0 and 1
random_idx=prcnadrp[idx_response].runif(seed=1234)
random_idx.head(10)

H2OFrame with 10 rows and 1 columns: 
        rnd
0  0.732572
1  0.651357
2  0.216731
3  0.588950
4  0.083199
5  0.299730
6  0.821594
7  0.918547
8  0.727564
9  0.358803


In [107]:
len(random_idx)

2844

In [95]:
#prcnadrp['target']=prcnadrp['listing price'].asnumeric
prcnadrp['property type'].unique()

H2OFrame with 59 rows and 1 columns: 
                    C1
0             Agri Biz
1         Agricultural
2               Anchor
3      Assisted Living
4        Community Ctr
5             Day Care
6      Distr Warehouse
7  Du / Tri / Fourplex
8        Econ. Service
9           Flex Space


In [108]:
#split into training and testing data set 
train_set=prcnadrp[random_idx<0.7]
test_set=prcnadrp[random_idx>=0.7]


In [109]:
#export data set into data frame (temp df)
temp_df=prcnadrp.as_data_frame()

#count number of True and False
num_false=temp_df.iloc[:,idx_response].value_counts()[0]
num_true=temp_df.iloc[:,idx_response].value_counts()[1]

#delete temp_df from memory 
del temp_df

#calculate ratio
ratio_true=float(num_true)/(num_false+num_true)

#print out of the ratio
print round(ratio_true,3)

0.46


In [110]:
# export data set into data frame(temp df)
temp_df_train=train_set.as_data_frame()
temp_df_test=test_set.as_data_frame()

#count number of True and False
num_false_train=temp_df_train.iloc[:,idx_response].value_counts()[0]
num_false_test=temp_df_test.iloc[:,idx_response].value_counts()[0]

num_true_train=temp_df_train.iloc[:,idx_response].value_counts()[1]
num_true_test=temp_df_test.iloc[:,idx_response].value_counts()[1]

#delete temp_df from memory
del temp_df_train 
del temp_df_test 

#calculate ratio
ratio_true_train=float(num_true_train)/(num_false_train +num_true_train)
ratio_true_test=float(num_true_test)/(num_false_test+num_true_test)

#print out the ratio
print round(ratio_true_train,3)
print round(ratio_true_test,3)

0.463
0.492


In [111]:
#holdout splits
#a vector with uniformly distributed numbers between 0 and 1 
random_idx=train_set[idx_response].runif()

#split the data set into two holdout sets 

train_holdout_hex=train_set[random_idx<0.8]
validation_holdout_hex=train_set[random_idx>=0.8]

#export data set into data frame (temp df)
temp_df_train=train_holdout_hex.as_data_frame()
temp_df_valid=validation_holdout_hex.as_data_frame()

#count number of True and False
num_false_train=temp_df_train.iloc[:,idx_response].value_counts()[0]
num_false_valid=temp_df_valid.iloc[:,idx_response].value_counts()[0]

num_true_train=temp_df_train.iloc[:,idx_response].value_counts()[1]
num_true_valid=temp_df_valid.iloc[:,idx_response].value_counts()[1]

#delete temp_df from memory 
del temp_df_train, temp_df_valid 

#calculate ratio 
ratio_true_train=float(num_true_train)/(num_false_train + num_true)
ratio_true_valid=float(num_true_valid)/(num_false_valid+num_true)

#print out the ratio 
print round(ratio_true_train,3)
print round(ratio_true_valid,3)

0.339
0.121


In [158]:
print features, len(features),'\n'
print response

[u'street address', u'city', u'state', u'zip code', u'listing price', u'property name', u'size sf', u'property class', u'cap rate'] 9 

property type


In [112]:
#GBM model development 
#range of grid search 
range_ntrees=np.arange(500,10000,100)
range_depth=np.arange(2,4,1)
range_min_rows=np.arange(10,20,1)

#sampling number
num_models=10

In [113]:
#define empty list to store the models 
gbm_models=[]

#loop over
for id_model in range(num_models):
    
    #choose mode parameters 
    ntree=random.sample(range_ntrees,1)[0]
    depth=random.sample(range_depth,1)[0]
    min_rows=random.sample(range_min_rows,1)[0]
    
    #build model 
    params_model={'x': features,
                  'y': response,
                  'training_frame':training_holdout_hex,
                  'validation_frame':validation_holdout_hex,
                  'ntrees':ntree,
                  'max_depth':depth,
                  'learn_rate':0.01
                  }
    gbm_model=h2o.gbm(**params_model)
    
    # store model 
    gbm_models.append(gbm_model)
                 


gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%


In [177]:
# ROC curve 
#gbm_models[0].model_performance()
test_set.head()

H2OFrame with 10 rows and 10 columns: 
                street address        city state  zip code listing price  \
0    2907 East South Boulevard  Montgomery    AL     36116    $2,945,000   
1            5860 Endeavor Way      Tanner    AL     35671    $5,749,800   
2        1524 East 10th Street    Anniston    AL     36207    $2,690,000   
3          5510 McFarland Blvd   Northport    AL     35476    $3,950,000   
4    2907 East South Boulevard  Montgomery    AL     36116    $2,945,000   
5  2608 West Meighan Boulevard     Gadsden    AL     35904    $4,406,000   
6          5510 McFarland Blvd   Northport    AL     35476    $3,950,000   
7                      Hwy 280     Chelsea    AL     35043      $995,000   
8           7033 Airport Blvd.      Mobile    AL     36608    $6,979,412   
9            5271 Endeavor Way  Huntsville    AL     35671   $15,514,000   

                                       property name    property type  \
0  One Center - $30MM Anchors Underway. 20% Final..

In [144]:
gbm_models[0].predict(test_set)

H2OFrame with 891 rows and 60 columns: 
           predict      Agri Biz  Agricultural        Anchor  Assisted Living  \
0    Free Std Bldg  6.392276e-05  3.866805e-23  2.977840e-32     2.996282e-32   
1  Distr Warehouse  5.876863e-10  7.330165e-27  2.736807e-37     1.763991e-36   
2             Mfg.  7.883240e-10  4.752531e-28  3.670137e-37     2.659947e-29   
3    Multifam Land  2.069746e-12  1.067677e-03  9.642668e-40     1.373532e-12   
4    Free Std Bldg  6.392276e-05  3.866805e-23  2.977840e-32     2.996282e-32   
5   Off. Warehouse  7.333598e-13  3.783031e-04  3.416624e-40     4.683481e-13   
6    Multifam Land  2.069746e-12  1.067677e-03  9.642668e-40     1.373532e-12   
7     Self Storage  7.694458e-12  4.639859e-30  3.582571e-39     7.628662e-37   
8     Other Retail  2.610557e-07  1.578925e-25  1.216127e-34     1.223479e-34   
9     Medical Off.  3.238099e-09  2.338959e-27  1.508555e-36     1.651890e-36   

   Community Ctr      Day Care  Distr Warehouse  Du / Tri / Fourplex

In [148]:
gbm_model.model_performance(test_set)


ModelMetricsMultinomial: gbm
** Reported on test data. **

MSE: 0.856424941996
R^2: 0.996082077923
LogLoss: 20.2525529062
    Agri Biz  Agricultural  Anchor  Assisted Living  Community Ctr  Day Care  \
0          0             2       0                0              0         0   
1          0             3       0                0              0         0   
2          0             0       0                0              0         0   
3          0             0       0                0              0         0   
4          0             1       0                0              0         0   
5          0             0       0                0              0         0   
6          0             1       0                0              0         0   
7          0             3       0                0              0         0   
8          0             0       0                0              0         0   
9          0             0       0                0              0         0 

In [146]:
gbm_model_roc=gbm_model.roc(valid=True)

AttributeError: 'H2OMultinomialModel' object has no attribute 'roc'

In [132]:
#plot(type='roc', train=False, valid=False, **kwargs)
#tnr(thresholds=None, train=False, valid=False)

NameError: name 'plot' is not defined

In [175]:
#auc(train=False, valid=False)
gbm_model.confusion_matrix(test_set)
#hit_ratio_table(train=False, valid=False)

    Agri Biz  Agricultural  Anchor  Assisted Living  Community Ctr  Day Care  \
0          0             2       0                0              0         0   
1          0             3       0                0              0         0   
2          0             0       0                0              0         0   
3          0             0       0                0              0         0   
4          0             1       0                0              0         0   
5          0             0       0                0              0         0   
6          0             1       0                0              0         0   
7          0             3       0                0              0         0   
8          0             0       0                0              0         0   
9          0             0       0                0              0         0   
..       ...           ...     ...              ...            ...       ...   
50         0             0       0      

In [174]:
#gbm_model.model_performance(valid=True)
#gbm_models[0].plot(type='roc', train=False, valid=False, **kwargs)

AttributeError: 'H2OMultinomialModel' object has no attribute 'plot'